In [1]:
import pandas as pd
import numpy as np

# Define sample data
data = {
    'unique_id': range(1, 13),
    'first_name': ['John', 'Jane', 'David', 'Emily', 'Michael', 'Sarah', 'John', 'Jane', 'David', 'Emily', 'John', 'John'],
    'last_name': ['Doe', 'Smith', 'Johnson', 'Brown', 'Davis', 'Miller', 'Doe', 'Smith', 'Johnson', 'Brown', 'Miller', 'Jones'],
    'full_name': ['John Doe', 'Jane Smith', 'David Johnson', 'Emily Brown', 'Michael Davis', 'Sarah Miller', 'John Doe', 'Jane Smith', 'David Johnson', 'Emily Brown', 'John Miller', 'John Jones'],
    'entity_type': ['Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person', 'Person'],
    'state': ['CA', 'NY', 'TX', 'FL', 'CA', 'NY', 'CA', 'TX', 'FL', 'NY', 'CA', 'FL'],
    'party': ['Democrat', 'Republican', 'Independent', 'Democrat', 'Republican', 'Independent', 'Democrat', 'Republican', 'Independent', 'Democrat', 'Republican', 'Independent'],
    'company': ['Apple', 'Google', 'Microsoft', 'Amazon', 'Facebook', 'Apple', 'Google', 'Microsoft', 'Amazon', 'Facebook', 'Google', 'Microsoft']
}

# Create DataFrame
df = pd.DataFrame(data)

/Users/naynapashilkar/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/naynapashilkar/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/var/folders/nk/h__9839s2k1_48m_z2g76vn40000gn/T/ipykernel_14716/3040970222.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466

In [2]:
from splink.duckdb.linker import DuckDBLinker
import altair as alt
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [3]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.last_name = r.last_name",
    ],
}
linker = DuckDBLinker(df, settings)

# linker.profile_columns(
#     ["first_name", "last_name"], top_n=10, bottom_n=5
# )

In [4]:
import splink.duckdb.comparison_template_library as ctl
import splink.duckdb.comparison_library as cl

settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        "l.first_name = r.first_name and l.last_name = r.last_name",
    ],
    "comparisons": [
        ctl.name_comparison("first_name", term_frequency_adjustments=True),
        ctl.name_comparison("last_name", term_frequency_adjustments=True),
        cl.exact_match("entity_type", term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("state", 2),
        cl.levenshtein_at_thresholds("party", 2),
        cl.levenshtein_at_thresholds("company", 2),
        ctl.name_comparison("full_name", term_frequency_adjustments=True),
        # Add more comparisons as needed
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    "max_iterations": 10,
    "em_convergence": 0.01
}

linker = DuckDBLinker(df, settings)


In [5]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.last_name = r.last_name",
        "l.full_name = r.full_name and l.state = r.state",
        "l.full_name = r.full_name and l.company = r.company",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.101.
This means that amongst all possible pairwise record comparisons, one in 9.90 are expected to match.  With 66 total possible comparisons, we expect a total of around 6.67 matching pairs


In [10]:
linker.estimate_u_using_random_sampling(max_pairs=5e6)

----- Estimating u probabilities using random sampling -----
u probability not trained for first_name - Damerau_levenshtein <= 1 (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
u probability not trained for first_name - Jaro_winkler_similarity >= 0.9 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained for first_name - Jaro_winkler_similarity >= 0.8 (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
u probability not trained for last_name - Damerau_levenshtein <= 1 (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
u probability not trained for last_name - Jaro_winkler_similarity >= 0.9 (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
u probability not trained fo

In [11]:
# linker.match_weights_chart()

In [12]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'first_name':
    u values not fully trained
Comparison: 'last_name':
    m values not fully trained
Comparison: 'last_name':
    u values not fully trained
Comparison: 'entity_type':
    m values not fully trained
Comparison: 'entity_type':
    u values not fully trained
Comparison: 'state':
    m values not fully trained
Comparison: 'state':
    u values not fully trained
Comparison: 'party':
    m values not fully trained
Comparison: 'party':
    u values not fully trained
Comparison: 'company':
    m values not fully trained
Comparison: 'company':
    u values not fully trained
Comparison: 'full_name':
    m values not fully trained
Comparison: 'full_name':
    u value

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,company_r,gamma_company,bf_company,full_name_l,full_name_r,gamma_full_name,tf_full_name_l,tf_full_name_r,bf_full_name,bf_tf_adj_full_name
0,-4.010330,0.058428,2,8,Jane,Jane,4,0.166667,0.166667,6.966667,...,Microsoft,0,0.028947,Jane Smith,Jane Smith,4,0.166667,0.166667,15.675,0.363636
1,-4.010330,0.058428,4,10,Emily,Emily,4,0.166667,0.166667,6.966667,...,Facebook,0,0.028947,Emily Brown,Emily Brown,4,0.166667,0.166667,15.675,0.363636
2,-4.010330,0.058428,3,9,David,David,4,0.166667,0.166667,6.966667,...,Amazon,0,0.028947,David Johnson,David Johnson,4,0.166667,0.166667,15.675,0.363636
3,2.265078,0.827788,1,7,John,John,4,0.333333,0.333333,6.966667,...,Google,0,0.028947,John Doe,John Doe,4,0.166667,0.166667,15.675,0.363636


In [13]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=0.7)
clusters.as_pandas_dataframe(limit=5)

Completed iteration 1, root rows count 0


,cluster_id,unique_id,first_name,last_name,full_name,entity_type,state,party,company,__splink_salt,tf_first_name,tf_last_name,tf_full_name,tf_entity_type
0,1,1,John,Doe,John Doe,Person,CA,Democrat,Apple,0.883414,0.333333,0.166667,0.166667,1.0
1,2,2,Jane,Smith,Jane Smith,Person,NY,Republican,Google,0.146300,0.166667,0.166667,0.166667,1.0
2,3,3,David,Johnson,David Johnson,Person,TX,Independent,Microsoft,0.355567,0.166667,0.166667,0.166667,1.0
3,4,4,Emily,Brown,Emily Brown,Person,FL,Democrat,Amazon,0.671364,0.166667,0.166667,0.166667,1.0
4,5,5,Michael,Davis,Michael Davis,Person,CA,Republican,Facebook,0.540680,0.083333,0.083333,0.083333,1.0
